# Scoring Analysis Notebook

This notebook imports all `calcular_` functions from `get_conver_scores.py` and displays dataframes for each metric.


In [7]:
import sys
import os
import pandas as pd
from IPython.display import display as iDisplay

# Add the scoring_scripts directory to the path
# From scoring_notebooks/, go up one level and into scoring_scripts/
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'scoring_scripts'))

# Alternative: if the above doesn't work, uncomment the line below
# sys.path.insert(0, '../scoring_scripts')

from get_conver_scores import (
    calcular_muletillas,
    calcular_claridad,
    calcular_participacion_dinamica,
    calcular_cobertura_temas,
    calcular_indice_preguntas,
    calcular_ppm_variabilidad, 
    get_conver_scores
)


## Input Transcript

Using the demo transcript from `get_conver_scores.py`


In [2]:
transcript_demo = [
    {
        "speaker": "vendedor", 
        "text": "Hola, buenas tardes. Bienvenido a nuestra exposición, eh... mi nombre es Carlos. Veo que se ha detenido justo delante del nuevo Conversa XL. Tiene buen ojo, es la unidad que acabamos de recibir esta misma mañana desde fábrica.", 
        "duracion": 18
    },
    {
        "speaker": "cliente", 
        "text": "Hola Carlos. Sí, la verdad es que estaba buscando algo más grande porque la familia ha crecido y mi coche actual se nos ha quedado minúsculo.", 
        "duracion": 12
    },
    {
        "speaker": "vendedor", 
        "text": "Le entiendo perfectamente. El espacio es vital. Déjeme decirle que hemos cambiado totalmente el **mindset** de diseño para enfocarnos en familias como la suya. Fíjese en estas líneas laterales, no solo son estética, son refuerzos de acero al boro que absorben impactos. De hecho, quería comentarle que en temas de seguridad somos líderes absolutos; hemos ganado los premios 'Safety Best 2024' y las 5 estrellas Euro NCAP, así que no tendrá ningún miedo al llevar a sus hijos en carretera.", 
        "duracion": 38
    },
    {
        "speaker": "cliente", 
        "text": "La seguridad es importante, claro. Pero mi esposa está obsesionada con el maletero. En el que tenemos ahora, meter el carrito del bebé y la compra es imposible, siempre tenemos que dejar bolsas en los asientos de atrás.", 
        "duracion": 18
    },
    {
        "speaker": "vendedor", 
        "text": "Comprendo. Si le sigo bien, lo que me está diciendo es que su mayor dolor de cabeza actual es la falta de capacidad de carga y necesita garantías de que podrá meter el carrito y las bolsas del supermercado todo junto en el maletero sin invadir los asientos, ¿es eso correcto?", 
        "duracion": 25
    },
    {
        "speaker": "cliente", 
        "text": "Exacto, eso es justo lo que necesito. Que no sea un tetris cada vez que salimos de viaje.", 
        "duracion": 8
    },
    {
        "speaker": "vendedor", 
        "text": "Pues venga por aquí, quiero que vea esto. Abra el portón. Tenemos 650 litros de capacidad real. O sea, aquí le caben dos carritos si hace falta. Además, la boca de carga es muy baja para que no se deje la espalda levantando peso. Y mire los asientos traseros, son individuales. No va a tener problema para colocar tres sillas infantiles, algo que muy pocos coches de la competencia permiten hoy en día.", 
        "duracion": 35
    },
    {
        "speaker": "cliente", 
        "text": "Oye, pues es verdad que se ve inmenso. ¿Y de tecnología qué tal va? Porque no quiero algo que sea muy complicado de usar, la pantalla esa parece una nave espacial.", 
        "duracion": 15
    },
    {
        "speaker": "vendedor", 
        "text": "Parece compleja, pero el **feedback** que recibimos de todos los usuarios es que se aprende a usar en cinco minutos. Mire, le voy a ser sincero, la integración tecnológica hoy en día es algo que no podemos ignorar de ninguna manera porque la conectividad nos facilita la vida y usted necesita saber que puede gestionar las llamadas y el mapa sin soltar el volante en ningún momento bajo ninguna circunstancia.", 
        "duracion": 40
    },
    {
        "speaker": "cliente", 
        "text": "Ya, mientras no se cuelgue... ¿Y el motor? Hago muchos kilómetros para ir al trabajo y la gasolina está carísima.", 
        "duracion": 10
    },
    {
        "speaker": "vendedor", 
        "text": "Esteee... no se preocupe por eso. Montamos un motor híbrido auto-recargable de última generación. El coche gestiona solo cuándo usar la batería y cuándo el motor térmico. Esto significa que en ciudad va a ir casi siempre en eléctrico, reduciendo el gasto de combustible a la mitad comparado con su coche actual. Es eficiencia pura.", 
        "duracion": 28
    },
    {
        "speaker": "cliente", 
        "text": "Suena bien lo del ahorro. Pero vamos a lo doloroso... he estado mirando el modelo similar de la marca alemana y se me va de precio. Imagino que este, siendo nuevo y con tanta tecnología, costará un ojo de la cara.", 
        "duracion": 18
    },
    {
        "speaker": "vendedor", 
        "text": "Para nada, y aquí es donde el Conversa XL realmente brilla. Sabemos que el **budget** familiar es sagrado. Hemos posicionado este vehículo con una estrategia muy agresiva. Si compara equipamiento por equipamiento, nuestro precio final está actualmente un 12% por debajo de la competencia directa alemana o japonesa. Básicamente, se lleva más coche por menos dinero.", 
        "duracion": 32
    },
    {
        "speaker": "cliente", 
        "text": "Un 12% es bastante diferencia... ¿Y tenéis financiación? Porque no quería descapitalizarme ahora mismo pagándolo todo de golpe.", 
        "duracion": 12
    },
    {
        "speaker": "vendedor", 
        "text": "Sí, tenemos un plan flexible. Bueno, podemos ajustar la entrada y dejar una cuota que ni note a fin de mes. Es la mejor solución para tener el coche ya sin agobios financieros.", 
        "duracion": 18
    },
    {
        "speaker": "cliente", 
        "text": "Tendría que consultarlo con mi mujer esta noche, no quiero precipitarme.", 
        "duracion": 8
    },
    {
        "speaker": "vendedor", 
        "text": "Es lógico, consúltelo. Pero déjeme advertirle de una cosa: esta campaña de lanzamiento con el descuento del 12% termina estrictamente este viernes. Si lo deja pasar, corre el riesgo de tener que pagar la tarifa oficial la semana que viene, que son casi 3.000 euros más. ¿Sabes? yo en su lugar dejaría hecha una reserva simbólica hoy, que es totalmente reembolsable, solo para bloquear el precio y las condiciones.", 
        "duracion": 40
    },
    {
        "speaker": "cliente", 
        "text": "Si es reembolsable... supongo que no pierdo nada por dejarlo reservado mientras lo pienso.", 
        "duracion": 10
    }
]


## Calculate All Metrics


In [3]:
# Calculate all metrics
muletillas_result = calcular_muletillas(transcript_demo)
claridad_result = calcular_claridad(transcript_demo)
participacion_result = calcular_participacion_dinamica(transcript_demo)
cobertura_temas_result = calcular_cobertura_temas(transcript_demo)
preguntas_result = calcular_indice_preguntas(transcript_demo)
ppm_result = calcular_ppm_variabilidad(transcript_demo)


-------------- Palabras negativas ----------------
--------------- Frases largas -----------------
------------------ Anglicismos ------------------
--------------------- Positivas ----------------------


## DataFrames for Each Metric


In [6]:
df_muletillas = pd.DataFrame([{
    "puntuacion": muletillas_result["puntuacion"], 
    "penalizacion": muletillas_result["penalizacion"],
    "total_muletillas": muletillas_result["total_muletillas"],
    "repeticion_constante": muletillas_result.get("repeticion_constante", None),
    "porcentaje_repeticion": muletillas_result.get("porcentaje", None),
    "muletillas_detectadas": muletillas_result["muletillas_usadas"]
}])

print("\n### 1. Muletillas \n")

pd.set_option("display.max_colwidth", None)

display(df_muletillas)

df_claridad = pd.DataFrame([{
    "puntuacion": claridad_result["puntuacion"],
    "penalizacion": claridad_result["penalizacion"],
    "penalizacion_negativas": claridad_result["penalizacion_negativas"], 
    "penalizacion_frases_largas": claridad_result["penalizacion_frases_largas"], 
    "penalizacion_anglicismos": claridad_result["penalizacion_anglicismos"], 
    "bonificacion": claridad_result["bonificacion"],
    "num_frases_largas": claridad_result["num_frases_largas"], 
    "positivas": claridad_result["positivas"], 
    "negativas": claridad_result["negativas"], 
    "anglicismos": claridad_result["anglicismos"],
    "frases_largas": claridad_result["frases_largas"]
}])

print("\n### 2. Claridad \n")
display(df_claridad)

# 2. Participación DataFrame
df_participacion = pd.DataFrame([{
    "puntuacion": participacion_result["puntuacion"],
    "bonificacion": participacion_result.get("bonificacion", None),
    "penalizacion": participacion_result.get("penalizacion", None),
    "palabras_cliente": participacion_result["palabras_cliente"],
    "palabras_vendedor": participacion_result["palabras_vendedor"],
    "ratio": participacion_result["ratio"],
    "escucha_activa": participacion_result["escucha_activa"],
    "n_escuchas": participacion_result["n_escuchas"]
}])
print("\n### 3. Participacion \n")
display(df_participacion)

# 3. Cobertura de Temas DataFrame
df_cobertura_temas = pd.DataFrame([{
    "puntuacion": cobertura_temas_result["puntuacion"],
    "temas_olvidados": cobertura_temas_result.get("temas_olvidados", None),
    "proximos_pasos": cobertura_temas_result.get("proximos_pasos", None),
    "penalizacion": cobertura_temas_result.get("penalizacion", None),
    "bonificacion": cobertura_temas_result.get("bonificacion", None),
    "señales_temas": cobertura_temas_result.get("señales_temas", None)
}])
print("\n### 4. Temas \n")
display(df_cobertura_temas)

# 4. Índice de Preguntas DataFrame
df_preguntas = pd.DataFrame([{
    "puntuacion": preguntas_result["puntuacion"],
    "total_preguntas": preguntas_result["total_preguntas"],
    "preguntas": preguntas_result["preguntas"],
    "cerradas": preguntas_result["cerradas"],
    "sondeo": preguntas_result["sondeo"],
    "irrelevantes": preguntas_result["irrelevantes"],
    "penalizacion": preguntas_result["penalizacion"],
    "bonificacion": preguntas_result["bonificacion"]
}])
print("\n### 5. Preguntas \n")
display(df_preguntas)

# 5. PPM DataFrame
df_ppm = pd.DataFrame([{
    "puntuacion": ppm_result["puntuacion"],
    "ppms": ppm_result["ppms"],
    "media_ppm": ppm_result["media_ppm"],
    "variabilidad": ppm_result["variabilidad"],
    "penalizacion": ppm_result["penalizacion"],
    "bonificacion": ppm_result["bonificacion"]
}])
print("\n### 6. PPM\n")
display(df_ppm)



### 1. Muletillas 



,puntuacion,penalizacion,total_muletillas,repeticion_constante,porcentaje_repeticion,muletillas_detectadas
0,0,110,22,False,0.227273,"[eh, se, si, pues, venga, si, se, mire, va, se, mire, esteee, se, va, este, si, se, bueno, ya, este, si, sabes]"



### 2. Claridad 



,puntuacion,penalizacion,penalizacion_negativas,penalizacion_frases_largas,penalizacion_anglicismos,bonificacion,num_frases_largas,positivas,negativas,anglicismos,frases_largas
0,0,135,45,45,45,30,3,"[entiendo, vital, seguridad, comprendo, capacidad, correcto, capacidad, eficiencia, solución, lógico]","[miedo, problema, ignorar, gasto, básicamente, riesgo]","[mindset, feedback, budget]","[de hecho, quería comentarle que en temas de seguridad somos líderes absolutos; hemos ganado los premios 'safety best 2024' y las 5 estrellas euro ncap, así que no tendrá ningún miedo al llevar a sus hijos en carretera, si le sigo bien, lo que me está diciendo es que su mayor dolor de cabeza actual es la falta de capacidad de carga y necesita garantías de que podrá meter el carrito y las bolsas del supermercado todo junto en el maletero sin invadir los asientos, ¿es eso correcto, mire, le voy a ser sincero, la integración tecnológica hoy en día es algo que no podemos ignorar de ninguna manera porque la conectividad nos facilita la vida y usted necesita saber que puede gestionar las llamadas y el mapa sin soltar el volante en ningún momento bajo ninguna circunstancia]"



### 3. Participacion 



...

,puntuacion,temas_olvidados,proximos_pasos,penalizacion,bonificacion,señales_temas
0,90,1,1,20,10,"[- Precio: Ubicación en la conversación del vendedor donde se menciona que el precio está por debajo de la competencia y la oferta de descuento: ""nuestro precio final está actualmente un 12% por debajo de la competencia directa alemana o japonesa"" y la mención de la campaña de lanzamiento con un descuento del 12% que finaliza este viernes., - Seguridad: Ubicación en la conversación del vendedor donde se destacan la seguridad y los premios: ""líneas laterales refuerzos de acero al boro que absorben impactos"" junto con ""hemos ganado los premios 'Safety Best 2024' y las 5 estrellas Euro NCAP""., - Espacio: Ubicación en la conversación del vendedor donde se aborda la capacidad y configuración para familias: apertura del portón, ""650 litros de capacidad real"" y que ""aquí le caben dos carritos si hace falta"", además de la boca de carga baja y asientos traseros individuales para colocar tres sillas infantiles., - Combustible: Ubicación en la conversación del vendedor donde se habla de eficiencia y motor híbrido: ""motor híbrido auto-recargable de última generación"" y que en ciudad irá casi siempre en eléctrico, reduciendo el gasto de combustible a la mitad., - Capacidad maletero: Ubicación en la conversación del vendedor donde se señala específicamente la capacidad del maletero: ""650 litros de capacidad real"" (referido como capacidad de maletero).]"



### 5. Preguntas 



,puntuacion,total_preguntas,preguntas,cerradas,sondeo,irrelevantes,penalizacion,bonificacion
0,100,2,"[¿es eso correcto?, ¿sabes?]",0,0,0,0,0



### 6. PPM



,puntuacion,ppms,media_ppm,variabilidad,penalizacion,bonificacion
0,40,"[126.66666666666667, 126.31578947368422, 122.39999999999999, 125.14285714285714, 105.0, 117.85714285714286, 105.0, 110.0, 103.5]",115.0,9.3,60,0


In [10]:

# Get scores for each KPI using the 'puntuacion' column from each DataFrame
kpi_scores = {
    'muletillas_pausas': df_muletillas['puntuacion'].iloc[0],
    'claridad': df_claridad['puntuacion'].iloc[0],
    'participacion': df_participacion['puntuacion'].iloc[0],
    'cobertura': df_cobertura_temas['puntuacion'].iloc[0],
    'preguntas': df_preguntas['puntuacion'].iloc[0],
    'ppm': df_ppm['puntuacion'].iloc[0]
}

# Define weights as in get_conver_scores.py
pesos = {
    "muletillas_pausas": 0.15,
    "claridad": 0.15,
    "participacion": 0.25,
    "cobertura": 0.15,
    "preguntas": 0.20,
    "ppm": 0.10
}

# Compute the weighted final score using the local DataFrame KPI scores
# Ensure the metric order matches the weights' keys
puntuacion_final = (
    kpi_scores['muletillas_pausas'] * pesos['muletillas_pausas'] +
    kpi_scores['claridad'] * pesos['claridad'] +
    kpi_scores['participacion'] * pesos['participacion'] +
    kpi_scores['cobertura'] * pesos['cobertura'] +
    kpi_scores['preguntas'] * pesos['preguntas'] +
    kpi_scores['ppm'] * pesos['ppm']
)

# Show rounded final score
print(f'Puntuacion final: {puntuacion_final:.1f}')


Puntuacion final: 55.0


## Summary

All metrics have been calculated and displayed in their respective DataFrames above.
